In [10]:
%pylab inline
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import copy
import numpy as np
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


C:\Users\leevi\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [11]:
filename = r'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
df = pd.read_csv(filename,
                index_col = None,
                header = None,
                na_values = '?')
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


# Using Neural networks to fill missing data values  
  
  ## Creating dataframes of Nan values and non Nan values
  

In [12]:
#marking all the Nan values with String 'nan'
df_nan = df.fillna('nan')

#creating empty dataframe
column_names = ([0,1,2,3,4,5,6,7,8,9,10,11,12,13])
nan_data_columns = pd.DataFrame(columns = column_names)
nan_data = pd.DataFrame(columns = column_names)

i = 0
#going through all the data rows
while i < df.shape[0]:
    contains_nan = 0
    a = 0
    
    #going through all the columns in a row
    for x in df_nan.loc[i,]:
        
        #if contains nan adding 1 to nan_data_columns so we know what columns have Nan values
        if x == 'nan':
            nan_data_columns.loc[1,a] = 1
            contains_nan = 1
        a = a + 1
        
    #if the row had Nan values adding it to nan_data dataframe    
    if contains_nan == 1:
        nan_data = nan_data.append(df_nan.loc[i, ])
    i = i + 1

print(nan_data)   
print(nan_data_columns)

#dropping Nan values so now df has all rows but the ones with Nan values
df = df.dropna()
df.head()

       0    1    2      3      4    5    6      7    8    9    10   11   12 13
87   53.0  0.0  3.0  128.0  216.0  0.0  2.0  115.0  0.0  0.0  1.0    0  nan  0
166  52.0  1.0  3.0  138.0  223.0  0.0  0.0  169.0  0.0  0.0  1.0  nan    3  0
192  43.0  1.0  4.0  132.0  247.0  1.0  2.0  143.0  1.0  0.1  2.0  nan    7  1
266  52.0  1.0  4.0  128.0  204.0  1.0  0.0  156.0  1.0  1.0  2.0    0  nan  2
287  58.0  1.0  2.0  125.0  220.0  0.0  0.0  144.0  0.0  0.4  2.0  nan    7  0
302  38.0  1.0  3.0  138.0  175.0  0.0  0.0  173.0  0.0  0.0  1.0  nan    3  0
    0    1    2    3    4    5    6    7    8    9    10 11 12   13
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1  1  NaN


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


## Creating Neural network for each column that has Nan values

In [13]:
i = 0

#going through all the columns
for x in nan_data_columns.loc[1,]:
    
    #if x = 1 that column has Nan values
    if x == 1:
        
        #data is all but the i column and making labels from the i column
        data = df.loc[:, df.columns != i]
        labels = 1.0*(df.loc[:, i] > 0)
        #print(data.head())
        #print(data.shape)
        #print(labels.shape)
        
        #creating the network structure
        network = Sequential()
        network.add(Dense(13, activation='relu', input_shape=(13,))),
        network.add(Dense(52, activation='relu')),
        network.add(Dense(1))
        network.summary()
        
        #compiling the network
        network.compile(optimizer='rmsprop',loss='mse',
              metrics=['mae'])
        
        #fitting with the non Nan values
        network.fit(data, labels, batch_size=10, epochs=50, verbose=1)
        
        #predicting the Nan values
        predict_data = copy.copy(nan_data)
        predict_data.drop(predict_data.loc[predict_data[i]!='nan'].index, inplace=True)
        predict_data_labels = predict_data.loc[:, i]
        
        #print(predict_data.loc[:, df.columns != i])
        #print(predict_data)
        predictions = network.predict(predict_data.loc[:, df.columns != i])
        
        y = 0
        #going through each row and setting the predicted value
        for row in predict_data.index:
            nan_data.loc[row,i] = predictions[y]
            if predictions[y] < 0:
                nan_data.loc[row,i] = 0
            y = y + 1
    i = i + 1
print(nan_data)

#adding the Nan data with the predicted values to the whole set
df.append(nan_data)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 13)                182       
_________________________________________________________________
dense_10 (Dense)             (None, 52)                728       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 53        
Total params: 963
Trainable params: 963
Non-trainable params: 0
_________________________________________________________________
Train on 297 samples
Epoch 1/50
297/297 [==============================] - 1s 2ms/sample - loss: 309.6536 - mae: 11.2659
Epoch 2/50
297/297 [==============================] - 0s 283us/sample - loss: 4.7866 - mae: 1.7066
Epoch 3/50
297/297 [==============================] - 0s 295us/sample - loss: 4.4071 - mae: 1.6918
Epoch 4/50
297/297 [==============================] - 0s 298us/samp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0,6,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3,3,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2,7,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0,3,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,52.0,1.0,3.0,138.0,223.0,0.0,0.0,169.0,0.0,0.0,1.0,0,3,0
192,43.0,1.0,4.0,132.0,247.0,1.0,2.0,143.0,1.0,0.1,2.0,0,7,1
266,52.0,1.0,4.0,128.0,204.0,1.0,0.0,156.0,1.0,1.0,2.0,0,0.937986,2
287,58.0,1.0,2.0,125.0,220.0,0.0,0.0,144.0,0.0,0.4,2.0,0,7,0


## Suffeling the data to merge data with generated values to the original data

In [14]:
#shuffeling the data and creating dataframe + labels
df = df.sample(frac=1)
data = df.loc[:, 0:12]
labels = 1.0*(df.loc[:, 13] > 0)

## Creting the training and test data

In [15]:
sets = np.split(data,3)
setlabels = np.split(labels,3)

training_data = sets[0].append(sets[1])
training_labels = setlabels[0].append(setlabels[1])
print(training_data.shape, training_labels.shape)
print(training_data.head())

test_data = sets[2]
test_labels = setlabels[2]
print(test_data.shape, test_labels.shape)
print(test_data.head())

(198, 13) (198,)
       0    1    2      3      4    5    6      7    8    9    10   11   12
57   41.0  1.0  4.0  110.0  172.0  0.0  2.0  158.0  0.0  0.0  1.0  0.0  7.0
265  42.0  1.0  4.0  136.0  315.0  0.0  0.0  125.0  1.0  1.8  2.0  0.0  6.0
9    53.0  1.0  4.0  140.0  203.0  1.0  2.0  155.0  1.0  3.1  3.0  0.0  7.0
252  64.0  1.0  4.0  128.0  263.0  0.0  0.0  105.0  1.0  0.2  2.0  1.0  7.0
133  51.0  1.0  4.0  140.0  261.0  0.0  2.0  186.0  1.0  0.0  1.0  0.0  3.0
(99, 13) (99,)
       0    1    2      3      4    5    6      7    8    9    10   11   12
270  61.0  1.0  4.0  140.0  207.0  0.0  2.0  138.0  1.0  1.9  1.0  1.0  7.0
251  58.0  1.0  4.0  146.0  218.0  0.0  0.0  105.0  0.0  2.0  2.0  1.0  7.0
198  50.0  0.0  2.0  120.0  244.0  0.0  0.0  162.0  0.0  1.1  1.0  0.0  3.0
177  56.0  1.0  4.0  132.0  184.0  0.0  2.0  105.0  1.0  2.1  2.0  1.0  6.0
23   58.0  1.0  3.0  132.0  224.0  0.0  2.0  173.0  0.0  3.2  1.0  2.0  7.0


In [16]:
network = Sequential()
network.add(Dense(13, activation='relu', input_shape=(13,))),
network.add(Dense(52, activation='relu')),
network.add(Dense(1, activation='sigmoid'))
network.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 13)                182       
_________________________________________________________________
dense_16 (Dense)             (None, 52)                728       
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 53        
Total params: 963
Trainable params: 963
Non-trainable params: 0
_________________________________________________________________


In [17]:
network.compile(optimizer='rmsprop',loss='binary_crossentropy',
              metrics=['accuracy'])
network.fit(training_data, training_labels, batch_size=10, epochs=150)

Train on 198 samples
Epoch 1/150
198/198 [==============================] - 1s 5ms/sample - loss: 1.7613 - accuracy: 0.5152
Epoch 2/150
198/198 [==============================] - 0s 332us/sample - loss: 1.2717 - accuracy: 0.5455
Epoch 3/150
198/198 [==============================] - 0s 360us/sample - loss: 1.2482 - accuracy: 0.4949
Epoch 4/150
198/198 [==============================] - 0s 340us/sample - loss: 1.0575 - accuracy: 0.5758
Epoch 5/150
198/198 [==============================] - 0s 381us/sample - loss: 0.9434 - accuracy: 0.6061
Epoch 6/150
198/198 [==============================] - 0s 376us/sample - loss: 0.9021 - accuracy: 0.6515
Epoch 7/150
198/198 [==============================] - 0s 356us/sample - loss: 0.9668 - accuracy: 0.5606
Epoch 8/150
198/198 [==============================] - 0s 396us/sample - loss: 1.0853 - accuracy: 0.5505
Epoch 9/150
198/198 [==============================] - 0s 331us/sample - loss: 0.9279 - accuracy: 0.5758
Epoch 10/150
198/198 [==============

In [18]:
test_loss, test_acc = network.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

99/99 [==============================] - 0s 2ms/sample - loss: 0.3060 - accuracy: 0.8687
Test accuracy: 0.86868685
